In [39]:
!pip install geopandas shapely folium


In [88]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import MousePosition, Fullscreen, MiniMap, MarkerCluster

# Функция для извлечения координат из geoData
def extract_coords(geo):
    if pd.isna(geo) or not isinstance(geo, str):
        return None, None
    try:
        coords = geo.split("[")[1].split("]")[0].split(",")
        return float(coords[1]), float(coords[0])
    except:
        return None, None

# ===== 1. Загрузка и обработка данных =====

# Аэропорты
aero = pd.read_csv("data-62883-2025-06-16.csv", sep=";", skiprows=1)
aero["Latitude_WGS84"], aero["Longitude_WGS84"] = zip(*aero["geoData"].map(extract_coords))
aero["name"] = aero["Наименование"]
aero["geometry"] = aero.apply(lambda row: Point(row["Longitude_WGS84"], row["Latitude_WGS84"]), axis=1)
aero_gdf = gpd.GeoDataFrame(aero, geometry="geometry", crs="EPSG:4326")

# Автовокзалы
bus = pd.read_csv("data-1881-2025-06-16.csv", sep=";", skiprows=1)
bus["Latitude_WGS84"], bus["Longitude_WGS84"] = zip(*bus["geoData"].map(extract_coords))
bus["name"] = bus["Наименование автовокзала"]
bus["geometry"] = bus.apply(lambda row: Point(row["Longitude_WGS84"], row["Latitude_WGS84"]), axis=1)
bus_gdf = gpd.GeoDataFrame(bus, geometry="geometry", crs="EPSG:4326")

# Ж/д вокзалы
train = pd.read_csv("data-62201-2025-06-16.csv", sep=";", skiprows=1)
train["Latitude_WGS84"], train["Longitude_WGS84"] = zip(*train["geoData"].map(extract_coords))
train["name"] = train["Наименование"]
train["geometry"] = train.apply(lambda row: Point(row["Longitude_WGS84"], row["Latitude_WGS84"]), axis=1)
train_gdf = gpd.GeoDataFrame(train, geometry="geometry", crs="EPSG:4326")

# Велопарковки
velo = pd.read_csv("data-916-2025-05-12.csv", sep=";", skiprows=1)
velo["Latitude_WGS84"], velo["Longitude_WGS84"] = zip(*velo["geoData"].map(extract_coords))
velo["name"] = velo["Наименование велосипедной парковки"]
velo["geometry"] = velo.apply(lambda row: Point(row["Longitude_WGS84"], row["Latitude_WGS84"]), axis=1)
velo_gdf = gpd.GeoDataFrame(velo, geometry="geometry", crs="EPSG:4326")

# ===== 2. Создание карты =====

m = folium.Map(location=[55.75, 37.62], zoom_start=10, control_scale=True)

# Плагины
MousePosition().add_to(m)
Fullscreen().add_to(m)
MiniMap().add_to(m)

# ===== 3. Слои =====

# Аэропорты
aero_layer = folium.FeatureGroup(name="Аэропорты", show=True)
for _, row in aero_gdf.iterrows():
    folium.Marker(
        location=[row["Latitude_WGS84"], row["Longitude_WGS84"]],
        popup=row["name"],
        icon=folium.Icon(color="blue", icon="plane", prefix="fa")
    ).add_to(aero_layer)
aero_layer.add_to(m)

# Автовокзалы
bus_layer = folium.FeatureGroup(name="Автовокзалы", show=True)
for _, row in bus_gdf.iterrows():
    folium.Marker(
        location=[row["Latitude_WGS84"], row["Longitude_WGS84"]],
        popup=row["name"],
        icon=folium.Icon(color="green", icon="bus", prefix="fa")
    ).add_to(bus_layer)
bus_layer.add_to(m)

# Ж/д вокзалы
train_layer = folium.FeatureGroup(name="Ж/д вокзалы", show=True)
for _, row in train_gdf.iterrows():
    folium.Marker(
        location=[row["Latitude_WGS84"], row["Longitude_WGS84"]],
        popup=row["name"],
        icon=folium.Icon(color="red", icon="train", prefix="fa")
    ).add_to(train_layer)
train_layer.add_to(m)

# Велопарковки (кластер)
velo_cluster = MarkerCluster(name="Велопарковки").add_to(m)
for _, row in velo_gdf.iterrows():
    folium.Marker(
        location=[row["Latitude_WGS84"], row["Longitude_WGS84"]],
        popup=row["name"],
        icon=folium.Icon(color="orange", icon="bicycle", prefix="fa")
    ).add_to(velo_cluster)

# ===== 4. Слои управления =====
folium.LayerControl(collapsed=False).add_to(m)

# ===== 5. Сохранение =====
m.save("moscow_transport_map.html")
